# Sesión 14 A

## Modelos de Mezcla Gaussiana (GMM)

> **Objetivos:**
>
> - Introducir K-Means como contraste simple a GMM.
> - Familiarizarse con el algoritmo Expectation-Maximization (EM) para GMM.
> - Explorar GMMs.

> **Lectura recomendada:**
>
> Mixture Models and EM. Bishop, C. M. (2006). Pattern Recognition and Machine Learning. Springer.

### 1. K-Means Clustering

#### 1.1. El modelo

Dado un conjunto de datos:

$$
X = \{x_1, x_2, \ldots, x_N\}, \qquad x_n \in \mathbb{R}^D
$$

Queremos particionarlos en $K$ clusters definidos por sus **centroides**:

$$
\mu_1, \mu_2, \ldots, \mu_K \in \mathbb{R}^D
$$

Introducimos variables de asignación:
$$
r_{nk} \in \{0,1\}
$$

donde  
- $r_{nk} = 1$ si el punto $x_n$ está asignado al cluster $k$  
- cada punto pertenece a un único cluster:
$$
\sum_{k=1}^K r_{nk} = 1
$$

#### 1.2. Función de costo (distorsión / inertia / WCSS)

La función objetivo que queremos minimizar es:

$$
J = \sum_{n=1}^N \sum_{k=1}^K r_{nk} \, \|x_n - \mu_k\|^2
$$

Esta mide la **suma de distancias cuadradas dentro del cluster**.  

Minimizar $J$ → significa clusters compactos.

> K-means es geométrico, no probabilístico.

#### 1.3. Algoritmo de optimización: algoritmo de Lloyd

K-means minimiza $J$ mediante un proceso iterativo de **descenso alternado**:

* **(A) Paso de asignación**
Fijando los centroides, asignamos cada punto al más cercano:

$$
r_{nk} =
\begin{cases}
1 & \text{si } k = \arg\min_j \|x_n - \mu_j\|^2 \\
0 & \text{otro caso}
\end{cases}
$$

* **(B) Paso de actualización de centroides**

Fijando las asignaciones, cada centroide se actualiza como el **promedio** de sus puntos:

$$
\mu_k =
\frac{
\sum_{n=1}^N r_{nk} \, x_n
}{
\sum_{n=1}^N r_{nk}
}
$$

* **(C) Criterio de convergencia**

El algoritmo termina cuando:
- las asignaciones no cambian, o  
- los centroides dejan de moverse, o  
- se alcanza un máximo de iteraciones.

🔥 <span style="color:#4f4559;">**Ejercicio en pizarron**</span>

![](../images/sesion14-img3.png)

**Figura 1:** (a) Los puntos verdes representan el conjunto de datos en un espacio euclidiano bidimensional. Las elecciones iniciales para los centros $\mu_1$ y $\mu_2$ se muestran con las cruces roja y azul, respectivamente.(b) En el paso **E** inicial, cada punto de datos se asigna al clúster rojo o al clúster azul, según cuál centroide esté más cerca. Esto es equivalente a clasificar los puntos según de qué lado de la **bisectriz perpendicular** entre los dos centros de clúster —mostrada por la línea magenta— se encuentren. (c) En el paso **M** posterior, cada centro de clúster se recalcula como la media de los puntos asignados al clúster correspondiente.(d)–(i) muestran los pasos E y M sucesivos hasta la convergencia final del algoritmo. Retomada de Bishop (2006).

#### 1.4. ¿Por qué se le llama **"hard-clustering"**?

Porque cada punto se asigna a un único cluster (asignación dura).

Anteriormente vimos en el paso de asignación:
$$
r_{nk} =
\begin{cases}
1 & \text{si } k = \arg\min_j \|x_n - \mu_j\|^2 \\
0 & \text{otro caso}
\end{cases}
$$

#### 1.5. ¿Qué asume el modelo K-means sobre los datos?

![](../images/sesion14-img1.png)

**Figura 2:** K-means asume que los clusters son esféricos y de tamaño similar, como ilustra la imagen de la izquierda. En la imagen de la derecha, K-means no puede capturar la estructura real de los datos debido a estas suposiciones. Retomada de Scikit-learn.

- Los clusters son esféricos (isotrópicos) y de tamaño similar.
- Fronteras de decisión lineales (basadas en distancia euclidiana).
- Forma uniforme (simétrica alrededor del centroide).
- Más puntos al centro del cluster que en los bordes.

> [Aquí](https://scikit-learn.org/stable/auto_examples/cluster/plot_cluster_comparison.html) puedes ver la documentación oficial de métodos de clustering en `scikit-learn`

```{admonition} Nota
:class: tip

K-means funciona bien si los clusters son:
- isotrópicos (no elípticos)
- convexos
- de tamaño parecido
- con densidad decreciente radialmente

Falla si los clusters son:
- elípticos
- rotados
- de distinta varianza
- no convexos
- solapados
```

![](../images/sesion14-img2.png)

**Figura 3:** Ejemplos de conjuntos de datos donde K-means puede fallar debido a sus suposiciones sobre la forma y distribución de los clusters. Retomada de Mael Fabien, 2020.

> * 💡 **¿Nos hemos preguntado por qué sucede lo de clústers esféricos?**

### 2. Gaussian Mixture Models (GMMs) y K-means

K-means es un método simple, rápido y muy útil cuando los clústers son “redondos”, de tamaño similar y bien separados. Pero estas suposiciones son fuertes y en muchos problemas reales **no se cumplen**.

Esto nos deja con dos limitaciones importantes:

1. **La forma del cluster está restringida.**  

2. **La asignación de los puntos es dura.**  

Ahora, planteemos una pregunta natural:

### _¿Y si pudiéramos relajar estas dos restricciones?_

<details>
<summary> Soft-clustering </summary>

- Para la forma:  
  **¿Qué define realmente la forma de un cluster?**  

- Para la asignación:  
  **¿Tiene sentido obligar a que cada punto pertenezca a un solo cluster?**

</details>

Estas dos ideas —modelar la forma y permitir asignaciones suaves— nos llevan directamente a los **Gaussian Mixture Models (GMMs)**.

Aquí van algunas pistas, si comparamos K-means y GMMs, tenemos que:

| k-Means | GMM (Gaussian Mixture Model) |
|--------|-------------------------------|
| Los clústers se definen por sus **medias** | Los clústers se definen por sus **medias y sus varianzas**, modelados como Gaussianas |
| Tiene limitaciones si los clústers están **superpuestos** | Funciona incluso si los clústers están **superpuestos** |
| Utiliza la **distancia euclidiana** al centroide | Utiliza la **probabilidad** de que X pertenezca a un clúster (modelo generativo) |


El algoritmo **k-Means** es, de hecho, un **caso especial** de un GMM con *Expectation-Maximization: Hard*, donde cada componente se modela mediante  

$$
\mathcal N(\mu_k, I),
$$

siendo $I$ la matriz identidad.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

In [ ]:
# Generemos dos clústers

np.random.seed(0)
n1 = 500
n2 = 500

#Clúster 1
mu1 = [0, 0]
cov1 =[[1,0],
       [0,1]]

C1 = np.random.multivariate_normal(mu1, cov1, n1)

#Clúster 2
mu2 = [5, 0]
cov2 =[[16,5],
       [4,12]]

C2 = np.random.multivariate_normal(mu2, cov2, n2)

In [ ]:
# Concatenar np.vstack
X = np.vstack([C1, C2])
X

In [ ]:
# Grafiquemos los datos
plt.figure(figsize=(10, 5))

plt.scatter(C1[:, 0], C1[:, 1], color='blue', alpha=0.5, label='Cluster 1 (Esférico)')
plt.scatter(C2[:, 0], C2[:, 1], color='red',  alpha=0.5, label='Cluster 2 (Elongado)')

plt.title("Clusters con covarianza esférica vs. covarianza no esférica")
plt.xlabel("X1")
plt.ylabel("X2")
plt.legend()
plt.grid(True)
plt.axis('equal')
plt.show()

In [ ]:
#import os
#os.environ["OMP_NUM_THREADS"] = "4"

In [ ]:
# Aplicar K-means
kmeans = KMeans(n_clusters=2, random_state=0)
labels = kmeans.fit_predict(X)
centers = kmeans.cluster_centers_

In [ ]:
# Graficar resultado de KMeans
plt.figure(figsize=(10, 5))

plt.scatter(X[:, 0], X[:, 1], c=labels, cmap='viridis', alpha=0.5)
plt.scatter(centers[:, 0], centers[:, 1], c='red', s=200, marker='X', label='Centroides')

plt.title("Resultado de KMeans sobre los clusters")
plt.xlabel("X1")
plt.ylabel("X2")
plt.legend()
plt.grid(True)
plt.axis('equal')
plt.show()

### 3. GMMs

Los GMMs son **modelos probabilísticos** que asumen que los datos son generados a partir de una **mezcla** de varias distribuciones Gaussianas.

#### 3.1. ¿Por qué una _"mezcla"_ de Gaussianas?

Pensemos en el siguiente ejemplo:

![](../images/sesion14-img4.png)

**Figura 4:** Ejemplo de un conjunto de datos que parece obvio que proviene de dos grupos distintos. Si modelamos con una sola distribucuión Gaussiana, quizá terminemos con un _promedio_ que no refleje la estructura real de los datos. Retomada de Mael Fabien, 2020.

#### 3.1. Set up

Tenemos datos $X = \{x_1, x_2, \ldots, x_N\}$ que parecen venir de varios grupos, pero:

* No sabemos cuántos grupos generaron los datos.
* No sabemos a qué grupo pertenece cada punto.
* No sabemos ni las medias ni las formas (var/cov) de esos grupos.

En un GMM, suponemos que:

> Cada grupo o componente es una distribución Gaussiana con sus propios parámetros (media y covarianza).

Para aprender el modelo, necesitamos 3 parámetros por componente:

* su media $\mu_k$
* su varianza/covarianza $\sigma_k$ o $\Sigma_k$
* su peso $w_k=p(z=k)$ (probabilidad de elegir ese grupo)

$$\theta = \{ \mu_k, \Sigma_k, w_k \}_{k=1}^K$$

> estos parámetros se **inicializan aleatoriamente** o con alguna otra técnica y se aprenden a partir de los datos usando el algoritmo de Expectation-Maximization (EM).

![](../images/sesion14-img6.png)

**Figura 5:** Se muestran tres distribuciones gaussianas (componentes de un modelo). Notese que cada gaussiana tiene un valor para sus parámetros $(\mu_k, \Sigma_k)$ y un peso $w_k$. 

#### 3.2. ¿Cómo nos ayuda la probabilidad y los grafos probabilísticos?

El objetivo principal de un Modelo de Mezcla Gaussiana (GMM) es poder calcular la _probabilidad de observar un dato $x$_ bajo el modelo:

$$
p(x)
\tag{1}
$$

> es decir, queremos saber qué tan probable es que el dato $x$ haya sido generado por nuestro modelo.

Pero hay un problema: 

* El dato $x$ sí lo observamos, 
* El componente del que proviene, $z$, **no lo observamos**.

Por eso introducimos un concepto importante: **la variable latente** $z_n$.

![](../images/sesion14-img5.png)


**Figura 6:** Diagrama de un modelo gráfico probabilístico para un GMM con $K$ componentes. Retomade de Bishop (2006).

**¿Qué representa la variable latente $z_n$?**

La variable $z_n$ es "latente" porque **no se observa** directamente. Su función es indicar a cuál componente del GMM pertenece el dato $x_n$.

En un modelo de mezclas gaussianas, esta relación suele representarse mediante un **grafo probabilístico** como el mostrado en la Figura 6.

En este grafo se expresa la distribución conjunta como:

$$
p(x,z)=p(z)p(x|z)
\tag{2}
$$

El nodo $z$ genera (o explica) al nodo $x$.

**¿cómo obtenemos $p(x) si $z$ no es observable?**

La _probabilidad total_ nos da  la clave: si hay variables ocultas, para obtener $p(x)$ debemos _sumar_ sobre todos los valores posibles de la variable oculta:

$$
p(x) = \sum_z p(x, z)
\tag{3}
$$

>Si no sé qué generó el dato $x$, considero todos los posibles orígenes.

Luego, si factorizamos la distribución conjunta usando la _regla de la cadena_:

$$
p(x,z) = p(z) \, p(x|z)\tag{4}
$$

y sustituimos, obtenemos:

$$
p(x) = \sum_z p(z) \, p(x|z)\tag{5}
$$

>Esta es la idea esencial! Aunque no separamos qué componente generó el dato, podemos considerar todos los componentes y ponderarlos por su probabilidad.

**Modelo GMM:**

En un GMM, los valores posibles de $z$ corresponde a los $K$ componentes de la mezcla.

Cada componente tiene: 

* un peso $w_k = p(z=k)$
* una distribución Gaussiana $p(x|z=k) = \mathcal{N}(x | \mu_k, \Sigma_k)$

Sustituyendo en la fórmula, obtenmos el modelo GMM:

$$ \boxed{p(x) = \sum_{k=1}^K w_k \, \mathcal{N}(x | \mu_k, \Sigma_k)}\tag{6}

El modelo en $(6)$ dice que la probabilidad de $x$ es una suma ponderada de varias Gaussianas: cada una aporta según qué tan probable es que ese componente sea el responsable de generar $x$.

#### 3.3. ¿De qué componente vino ese dato?

Hasta ahora sabemos cómo calcula un GMM la probabilidad total de un dato $x$. Pero el siguiente paso es más interesante:

* **Inferencia** de $Z$: queremos inferir de qué componente $k$ de la mezcla proviene un dato $x_i$.

> ¿Cuál es la probabilidad de que el dato $x_i$ haya sido generado por el componente $k$?

A esta probabilidad se le llama _resposability_ (responsabilidad) del componente $k$ sobre el dato $x_i$:

$$ 
\gamma_{zk} = p(z_k = k | x_i)\tag{7}
$$

Aplicamos el teorema de Bayes:

$$
\gamma_{zk} = \frac{p(z_k=k) \, p(x_i | z_k=k)}{p(x_i)}
\tag{8}
$$

> y esto tiene una lectura muy intuitiva:
>
> * $p(z=k)$: lo probable que es el componente $k$ antes de ver el dato (prior)
> * $p(x_i|z=k)$: qué tan bien explica el componente $k$ el dato.
> * $p(x_i)$: normaliza la probabilidad para sumar 1.

En la **Figura 6** imaginamos un dato $x_i$ y tres gaussianas distintas. Cada gaussiana tiene una **altura** diferente en el punto $x_i$.

> La altura de cada gaussiana en $x_i$ describe qué tan probable es ese dato si ese componente lo hubiere generado.

![](../images/sesion14-img7.png)

**Figura 6:** La figura representa un dato $x_i$. Hay 3 distribuciones gaussianas del modelo $k=1,2,3$. Cada gaussiana tiene un **valor distinto** en ese punto $x_i$. Cada dato está asociado (o no) a varias Gaussianas. **La altura relativa de cada Gaussiana en la posición del dato determina la probabilidad de que ese dato pertenezca a ese componente.** Esa probabilida es precisamente $\gamma_{ik} = p(z_k = k | x_i)$. Retomada de Mael Fabien, 2020.

Sustityendo cada término usando el modelo GMM:

* Prior (peso del componente): $p(z_k=k) = w_k$
* Likelihood (gaussiana del componente): $p(x_i | z_k=k) = \mathcal{N}(x_i | \mu_k, \Sigma_k)$
* Evidencia (prob. total del dato): $p(x_i) = \sum_{c=1}^K w_c \, \mathcal{N}(x_i | \mu_c, \Sigma_c)$


$$
\gamma_{ik} = \frac{w_k \, \mathcal{N}(x_i | \mu_k, \Sigma_k)}{\sum_{c=1}^K w_c \, \mathcal{N}(x_i | \mu_c, \Sigma_c)}\tag{9}


#### 3.4. Algoritmo Expectation-Maximization (EM)

Esto nos lleva de forma natural al algoritmo EM, un método elegante y poderoso para encontrar soluciones de máxima verosimilitud en modelos con variables latentes.

El algoritmo EM alterna dos pasos: **Estimación** y **Maximización**.
  
La idea general puede visualizarse como en la **Figura 7**:

![](../images/sesion14-img8.png)

**Figura 7:** Diagrama del algoritmo Expectation-Maximization (EM) para Gaussian Mixture Models (GMMs). Retomada de Mael Fabien, 2020.

- Partimos de unos **parámetros iniciales** $\theta^{(t)}$ del modelo.

- En el **E-step (Estimation)**, mantenemos fijos esos parámetros y calculamos las  $\gamma_{ik}^{(t)}$, es decir, la probabilidad de que cada dato haya sido generado por cada componente.

- En el **M-step (Maximization)**, mantenemos fijas las  $\gamma_{ik}^{(t)}$ y actualizamos los parámetros del modelo para obtener $\theta^{(t+1)}$.

* **E-step:**

En el algoritmo EM, cada iteración tiene parámetros actuales 

$$\theta^{(t)} = \{\, w_k^{(t)},\, \mu_k^{(t)},\, \Sigma_k^{(t)} \,\}$$

El **E-step** consiste precisamente en **evaluar la expresión anterior usando los parámetros actuales**, es decir:

$$
\gamma_{ik}^{(t)}
=
p(z_i = k \mid x_i, \theta^{(t)})
=
\frac{
w_k^{(t)} \, \mathcal{N}(x_i \mid \mu_k^{(t)}, \Sigma_k^{(t)})
}{
\sum_{c=1}^{K} 
w_c^{(t)} \, \mathcal{N}(x_i \mid \mu_c^{(t)}, \Sigma_c^{(t)})
}
$$

Estas probabilidades $\gamma_{ik}^{(t)}$ son la **salida del E-step** y se usarán como “asignaciones suaves” en el siguiente paso, el **M-step**, para actualizar los parámetros del modelo.

> E-step calculamos $\gamma_{ik} = p(z_k = k | x_i, \theta^{(t)})$ usando los parámetros actuales $\theta^{(t)}$.

* **M-step:** 
 
Una vez calculadas las $\gamma_{ik}^{(t)}$ en el E-step, el **M-step** mantiene fijas estas probabilidades y actualiza los parámetros del modelo para obtener nuevos valores

$$\theta^{(t+1)} = \{\, w_k^{(t+1)},\, \mu_k^{(t+1)},\, \Sigma_k^{(t+1)} \,\}$$

En esta etapa, cada dato contribuye a los parámetros de cada componente de forma **ponderada** por su $\gamma_{ik}^{(t)}$.

El resultado es equivalente a realizar una **estimación de máxima verosimilitud**, pero donde cada dato tiene un peso diferente según qué componente lo explica mejor.

Las actualizaciones toman la forma:

- Nuevos pesos:
  $$
  w_k^{(t+1)} = \frac{1}{N}\sum_{i=1}^N \gamma_{ik}^{(t)}
  $$

- Nuevas medias:
  $$
  \mu_k^{(t+1)} =
  \frac{\sum_{i=1}^N \gamma_{ik}^{(t)} \, x_i}{\sum_{i=1}^N \gamma_{ik}^{(t)}}
  $$

- Nuevas covarianzas:
  $$
  \Sigma_k^{(t+1)} =
  \frac{
    \sum_{i=1}^N \gamma_{ik}^{(t)}
    (x_i - \mu_k^{(t+1)})(x_i - \mu_k^{(t+1)})^\top
  }{
    \sum_{i=1}^N \gamma_{ik}^{(t)}
  }
  $$

Con estos parámetros actualizados $\theta^{(t+1)}$, comienza una nueva iteración del algoritmo: volvemos al E-step, calculamos nuevas $\gamma_{ik}^{(t+1)}$, y repetimos el ciclo hasta que el modelo converge.

![](../images/sesion14-img9.png)

**Figura 8:** Visualización del proceso iterativo del algoritmo Expectation-Maximization (EM) para Gaussian Mixture Models (GMMs). En cada iteración, el E-step calcula las responsabilidades $\gamma_{ik}$ basadas en los parámetros actuales del modelo, y el M-step actualiza los parámetros utilizando estas responsabilidades. Este ciclo se repite hasta la convergencia del modelo.

> Notas para revisar a fondo el algoritmo EM:
> - [Mixture Models and EM. Bishop, C. M. (2006). Pattern Recognition and Machine Learning. Springer.](https://www.microsoft.com/en-us/research/wp-content/uploads/2006/01/Bishop-Pattern-Recognition-and-Machine-Learning-2006.pdf)
> - [Columbia University](https://www.columbia.edu/~mh2078/MachineLearningORFE/EM_Algorithm.pdf)
> - [Wiki](https://en.wikipedia.org/wiki/Expectation%E2%80%93maximization_algorithm)
> - [EM-GMM-HMM](https://github.com/maelfabien/EM_GMM_HMM?tab=readme-ov-file)